In [1]:
import numpy as np
import pandas as pd

A = 0.1*np.array([[np.sqrt(99), -1], [1, np.sqrt(99)]])
B = 0.5*np.array([[np.sqrt(2), -2], [np.sqrt(2), np.sqrt(2)]])

sigma1 = .01
sigma2 = .2

X_0 = np.array([[1,0]]).T
Y_0 = np.dot(B, X_0) + np.random.normal(loc=0, scale=sigma2)

In [2]:
process_length = 2000

X = [X_0]
Y = [Y_0]

for _ in range(process_length):
    X_k = X[-1]
    X_k_next = np.dot(A, X_k) + np.random.normal(loc=0, scale=sigma1)    
    Y_k_next = np.dot(B, X_k_next) + np.random.normal(loc=0, scale=sigma2)
    X.append(X_k_next)    
    Y.append(Y_k_next)

def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data_utils

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 4
input_size = 2
hidden_size = 12
num_layers = 1
num_classes = 2
batch_size = 1
num_epochs = 5
learning_rate = 0.001

Y_seq, Y_target = sliding_windows(Y, sequence_length)
    
Y_seq = Variable(torch.Tensor(Y_seq.reshape(-1, sequence_length, 1, 2)))
Y_target = Variable(torch.Tensor(Y_target.reshape(-1, 1, 2)))

train_samples = 1000

train_tensor = data_utils.TensorDataset(Y_seq[:train_samples], Y_target[:train_samples]) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size)
test_tensor = data_utils.TensorDataset(Y_seq[-train_samples:], Y_target[-train_samples:]) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size)

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.last_hidden = None
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, hidden = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        self.last_hidden = hidden
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(train_loader)
# Train the model
for epoch in range(num_epochs):
    for i, (Y_seq_i, Y_target_i) in enumerate(train_loader):
        Y_seq_i = Y_seq_i.reshape(-1, sequence_length, input_size).to(device)
        Y_target_i = Y_target_i.reshape(-1, num_classes).to(device)
        # Forward pass
        outputs = model(Y_seq_i)
        loss = criterion(outputs, Y_target_i)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/1000], Loss: 0.6397
Epoch [1/5], Step [200/1000], Loss: 0.2281
Epoch [1/5], Step [300/1000], Loss: 0.0689
Epoch [1/5], Step [400/1000], Loss: 0.0112
Epoch [1/5], Step [500/1000], Loss: 0.0139
Epoch [1/5], Step [600/1000], Loss: 0.0104
Epoch [1/5], Step [700/1000], Loss: 0.0354
Epoch [1/5], Step [800/1000], Loss: 0.0382
Epoch [1/5], Step [900/1000], Loss: 0.1326
Epoch [1/5], Step [1000/1000], Loss: 0.0556
Epoch [2/5], Step [100/1000], Loss: 0.1849
Epoch [2/5], Step [200/1000], Loss: 0.0434
Epoch [2/5], Step [300/1000], Loss: 0.0164
Epoch [2/5], Step [400/1000], Loss: 0.0212
Epoch [2/5], Step [500/1000], Loss: 0.0012
Epoch [2/5], Step [600/1000], Loss: 0.0028
Epoch [2/5], Step [700/1000], Loss: 0.0306
Epoch [2/5], Step [800/1000], Loss: 0.0371
Epoch [2/5], Step [900/1000], Loss: 0.1316
Epoch [2/5], Step [1000/1000], Loss: 0.0427
Epoch [3/5], Step [100/1000], Loss: 0.1963
Epoch [3/5], Step [200/1000], Loss: 0.0468
Epoch [3/5], Step [300/1000], Loss: 0.0146
Epoch [3/

In [18]:
model.eval()
Y1001_hat = model(Y_seq[1001])

In [19]:
Y1001_hat.data

tensor([[0.1527, 0.3436],
        [0.1861, 0.5049],
        [0.0521, 0.4363],
        [0.0834, 0.5742]])

In [22]:
Y_target[(1001-sequence_length): 1001]

tensor([[[0.7191, 0.6367]],

        [[1.0000, 1.1544]],

        [[0.5310, 0.9117]],

        [[0.7779, 1.3918]]])

In [ ]:
model.last_hidden[0]

In [ ]:
X[1001]